In [1]:
!pip install bionumpy
!wget https://raw.githubusercontent.com/bionumpy/bionumpy/dev/example_data/ctcf_chr21-22.bed.gz
!wget https://raw.githubusercontent.com/bionumpy/bionumpy/dev/example_data/CTCF_chr21-22.wig.gz
!wget https://raw.githubusercontent.com/bionumpy/bionumpy/dev/example_data/chr21-22.chrom.sizes
!wget https://raw.githubusercontent.com/bionumpy/bionumpy/dev/example_data/ctcf_chr21-22.bam
!wget https://raw.githubusercontent.com/bionumpy/bionumpy/dev/example_data/chr21a22.gtf
!wget https://raw.githubusercontent.com/bionumpy/bionumpy/dev/example_data/1000Genomes_chr21-22.vcf.gz

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 161.4/161.4 kB 1.0 MB/s eta 0:00:00
--2024-06-30 17:18:01--  https://raw.githubusercontent.com/bionumpy/bionumpy/dev/example_data/ctcf_chr21-22.bed.gz
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 25694 (25K) [application/octet-stream]
Saving to: ‘ctcf_chr21-22.bed.gz’

ctcf_chr21-22.bed.g 100%[===================>]  25.09K  --.-KB/s    in 0s      

2024-06-30 17:18:01 (136 MB/s) - ‘ctcf_chr21-22.bed.gz’ saved [25694/25694]

--2024-06-30 17:18:01--  https://raw.githubusercontent.com/bionumpy/bionumpy/dev/example_data/CTCF_chr21-22.wig.gz
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.111.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubuse

In [2]:
import plotly.express as px
import numpy as np
import bionumpy as bnp

In [3]:
import bionumpy as bnp
import plotly.express as px
import numpy as np

# Reading a genome and reads from a bam file
genome = bnp.Genome.from_file("chr21-22.chrom.sizes")
reads = genome.read_intervals("ctcf_chr21-22.bam")

# Getting read pileup (stored efficiently as a RunLengthArray)
pileup = reads.get_pileup()

# We can index any region
region = pileup["chr22"][19970400:19970800]
px.line(region.to_array()).show()

In [7]:
# Read genome, a wig read pileup and transcripts
genome = bnp.Genome.from_file("chr21-22.chrom.sizes", sort_names=True)
annotation = genome.read_annotation("chr21a22.gtf")
transcripts = annotation.transcripts

# Get transcript start locations and make windows around them
tss = transcripts.get_location('start').sorted() # Make sure the transcripts are sorted alphabetically
windows = tss.get_windows(flank=500)

# Get mean read pileup within these windows and plot
track = genome.read_track("CTCF_chr21-22.wig.gz", stream=True)
signals = track[windows]
mean_signal = signals.mean(axis=0)
signal = bnp.compute(mean_signal)  # Compute the actual value

px.line(x=np.arange(-500, 501), y=signal.to_array(),
        title="Read pileup relative to TSS start",
        labels={"x": "Position relative to TSS start", "y": "Mean read pileup"}).show()

